# Pytorch Lightning Tutorial
This tutorial demonstrates integration of ModelBox with PyTorch Lightning.

In [1]:
import os

import pandas as pd
import seaborn as sn
import torch
import random
from IPython.core.display import display
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import MNIST
from modelbox.lightning_logger import ModelBoxLogger

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
BATCH_SIZE = 256 if torch.cuda.is_available() else 64

/home/diptanuc/Src/miniconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
/tmp/ipykernel_172281/3666529220.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [2]:
class MNISTModel(LightningModule):
    def __init__(self, data_dir=PATH_DATASETS, hidden_size=64, learning_rate=2e-4):
        super().__init__()
        self.l1 = torch.nn.Linear(28 * 28, 10)
        
        self.data_dir = data_dir
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate
        self.transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,)),
            ]
        )
        self.val_accuracy = Accuracy()
        self.test_accuracy = Accuracy()
        self.save_hyperparameters()

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.val_accuracy.update(preds, y)

        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.val_accuracy, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.test_accuracy.update(preds, y)

        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", self.test_accuracy, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

    def prepare_data(self):
        MNIST(self.data_dir, train=True, download=True)
        MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        if stage == "fit" or stage is None:
            mnist_full = MNIST(self.data_dir, train=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=BATCH_SIZE)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=BATCH_SIZE)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=BATCH_SIZE)



In [3]:
mnist_model = MNISTModel()

train_ds = MNIST(PATH_DATASETS, train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)

# Intialize ModelBoxLogger
experiment_name = f"lid_quartznet-{random.randint(1, 10000)}"
mbox_logger = ModelBoxLogger("langtech", experiment_name, "owner@pytorch.com")

trainer = Trainer(
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None, 
    max_epochs=3,
    logger=[mbox_logger],
    callbacks=[TQDMProgressBar(refresh_rate=20)],
)

trainer.fit(mnist_model, train_loader)
trainer.test()


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
modelbox - attempting to create a project
modelbox - created experiment with id: af724ad2dd984d646c0af24fd0a42bc6c9c6c745

  | Name          | Type     | Params
-------------------------------------------
0 | l1            | Linear   | 7.9 K 
1 | val_accuracy  | Accuracy | 0     
2 | test_accuracy | Accuracy | 0     
-------------------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)
modelbox - attempting to create a project
modelbox - created experiment with id: af724ad2dd984d646c0af24fd0a42bc6c9c6c745
modelbox - log hpraams params "data_dir":      .
"hidden_size":   64
"learning_rate": 0.0002
modelbox - log hpraams metrics None


Sanity Checking: 0it [00:00, ?it/s]

/home/diptanuc/Src/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/diptanuc/Src/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

modelbox - log metrics, step: 2813, key: val_loss, metrics: MetricValue(step=2813, wallclock_time=1659596682, value=29.673599243164062)
modelbox - log metrics, step: 2813, key: val_acc, metrics: MetricValue(step=2813, wallclock_time=1659596682, value=0.5217999815940857)
/home/diptanuc/Src/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1444: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  rank_zero_warn(
Restoring states from the checkpoint path at /home/diptanuc/Projects/modelbox/tutorials/lid_quartznet-9345/0.1/checkpoints/epoch=2-step=2814.ckpt
modelbox - attempting to create a project
modelbox - created experiment with id: af724ad2dd984d646c0af24fd0a42bc6c9c6c745
modelbox - log hpraams params "data_dir":    

Testing: 0it [00:00, ?it/s]

modelbox - log metrics, step: 2814, key: test_loss, metrics: MetricValue(step=2814, wallclock_time=1659596683, value=28.46238136291504)
modelbox - log metrics, step: 2814, key: test_acc, metrics: MetricValue(step=2814, wallclock_time=1659596683, value=0.531000018119812)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.531000018119812     │
│         test_loss         │     28.46238136291504     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 28.46238136291504, 'test_acc': 0.531000018119812}]

In [5]:
from modelbox.modelbox import ModelBoxClient
client = ModelBoxClient(addr="localhost:8085")


Once an experiment is logged into ModelBox from a lightning trainer, it can then be accessed via the client API.

In [10]:
client.list_metadata('af724ad2dd984d646c0af24fd0a42bc6c9c6c745').metadata

{'hyperparams': fields {
   key: "data_dir"
   value {
     string_value: "."
   }
 }
 fields {
   key: "hidden_size"
   value {
     number_value: 64.0
   }
 }
 fields {
   key: "learning_rate"
   value {
     number_value: 0.0002
   }
 }}

In [9]:
client.get_metrics('af724ad2dd984d646c0af24fd0a42bc6c9c6c745')

{'val_loss': Metrics(key='val_loss', values=[[MetricValue(step=937, wallclock_time=1659596673, value=16.853092193603516), MetricValue(step=1875, wallclock_time=1659596677, value=23.811155319213867), MetricValue(step=2813, wallclock_time=1659596682, value=29.673599243164062)]]),
 'val_acc': Metrics(key='val_acc', values=[[MetricValue(step=937, wallclock_time=1659596673, value=0.6092000007629395), MetricValue(step=1875, wallclock_time=1659596677, value=0.5605999827384949), MetricValue(step=2813, wallclock_time=1659596682, value=0.5217999815940857)]]),
 'test_acc': Metrics(key='test_acc', values=[[MetricValue(step=2814, wallclock_time=1659596683, value=0.531000018119812)]]),
 'test_loss': Metrics(key='test_loss', values=[[MetricValue(step=2814, wallclock_time=1659596683, value=28.46238136291504)]])}